In [1]:
import sys
sys.path.append("..")
sys.path.append("../dl")

import torch
import segmentation_models_pytorch as smp
from pytorch_lightning.logging import TestTubeLogger
from pytorch_lightning.callbacks import ModelCheckpoints
from config import user_conf, get_conf
from lightning_model import *

/home/leos/.local/lib/python3.6/site-packages/pandas/compat/_optional.py:106: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.2.0' currently installed).
  warnings.warn(msg, UserWarning)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

In [2]:
model = smp.FPN(
    encoder_name="resnext50_32x4d", 
    classes=2, 
    decoder_merge_policy='cat'
)

config = get_conf(user_conf, "pannuke")
lightning_model = SegModel.from_conf(model, config)

In [4]:
tt_logger = TestTubeLogger(
    save_dir=config['experiment_args']['experiment_root_dir'],
    name=config['experiment_args']['model_name'],
    version=config['experiment_args']['experiment_version']
)

checkpoint_dir = (
    Path(tt_logger.save_dir)
    / tt_logger.experiment.name
    / f"version_{tt_logger.experiment.version}"
)

checkpoint_callback = ModelCheckpoint(
    filepath = str(checkpoint_dir),
    save_top_k = 1,
    save_last = True,
    verbose = True, 
    monitor = 'avg_val_loss',
    mode = 'min',
    prefix = ''
)

if config["training_args"]['resume_training']:   
    last_checkpoint_path = lightning_model.fm.model_checkpoint("last")
    trainer = pl.Trainer(
        default_root_dir=config['experiment_args']['experiment_root_dir'],
        max_epochs=config['training_args']['num_epochs'], 
        gpus=config['training_args']['num_gpus'],  
        logger=tt_logger,
        checkpoint_callback=checkpoint_callback,
        resume_from_checkpoint=str(last_checkpoint_path)
    )
    
else:
    trainer = pl.Trainer(
        default_root_dir=config['experiment_args']['experiment_root_dir'],
        max_epochs=config['training_args']['num_epochs'], 
        gpus=config['training_args']['num_gpus'],  
        logger=tt_logger,
        checkpoint_callback=checkpoint_callback,
    )

/home/leos/.local/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning:

Checkpoint directory ../../results/tests/FPN/version_test_pannuke3 exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [5]:
log_dir = (
    Path(tt_logger.save_dir)
    / tt_logger.experiment.name
    / f"version_{tt_logger.experiment.version}"
    / "tf"
)

In [6]:
%load_ext tensorboard
%tensorboard --logdir {log_dir}

Reusing TensorBoard on port 6006 (pid 8068), started 0:04:44 ago. (Use '!kill 8068' to kill it.)

In [ ]:
trainer.fit(lightning_model)


  | Name  | Type             | Params
-------------------------------------------
0 | model | FPN              | 25 M  
1 | CE    | CrossEntropyLoss | 0     


In [ ]:
plot_metrics(conf=conf, metric='accuracy', scale='linear', save=True)